# Convert from TEI to TF to WATM

We convert Suriano TEI to TF and then to WATM.

This notebook provides three levels of refinement in the execution. They all have the same outcome,
but they differ in the level of detail they provide on the conversion.

These are the levels:

* **Express**: one single command on the command line for the complete conversion;
* **Step by step**: one command for each main step of the conversion;
* **Debugging**: all the commands directly in Python, the intermediate data remains in memory and can be inspected.

# Production or development

Mosts steps are unaffected by the production/development setting.

In the first steps of the pipeline (*ingest* and *scan processing*) we prepare both the dev and the prod data.

The intermediate steps (*from DOCX to TEI*, *from TEI to TF*, *mark named entities*) are identical for prod and dev.

Only for the latter steps (*convert TF to WATM*, generate IIIF manifests*, *deploy to k8s*) there is a distinction between prod and dev.

For these steps we have commented out the line that does the dev version.

# Requirements

* zsh as command line shell (as in macOS);
* access to suitable k8s clusters, streamlined by the 
  [k-suite](https://code.huc.knaw.nl/tt/smart-k8s/-/blob/main/docs/k-suite.md);
* [Pandoc](https://pandoc.org)
* [Imagemagick](https://imagemagick.org)
* [Python](https://www.python.org) (3.12 or higher) with additional pip-installable modules:
  * text-fabric
  * doc2python
  * openpyxl

# Declare the version

Always set the version before running any cell in this notebook!

In [1]:
VERSION = "0.4.3"

# Express: One shot

Here is the express, mindless way to convert the corpus.
If something goes wrong, you can follow the step-by-step section or the debugging section.

In [17]:
%%time

!python make.py all $VERSION
# !python make.py all $VERSION --no-prod

DOCX ==> TEI files ...
DOCX => simple TEI per filza ...
DOCX => headers ...
simple TEI per filza => enriched TEI per letter ...
	There were 5234 page warnings.
	There were 541 warnings.
	Pages with    transcription and no scan:  4675
	Pages with no transcription and    scan:   331
TEI => TF ...
	Validating TEI ...
	Converting TEI ...
	Loading TF ...
Annotate named entities ...
	Loading TF  ...
	489 entities targeted with 6220 occurrences. See ~/gitlab.huc.knaw.nl/suriano/letters/_temp/ner/0.4.1/.people.0.5/hits.tsv
	Loading TF with entities ...
TF => WATM ...
	Loading TF ...
	Making WATM for version 0.4.1e
	Writing WATM ...
	Testing WATM ...
	OK - whether all tests passed
Process images ...
Generate IIIF manifests ...
CPU times: user 623 ms, sys: 219 ms, total: 842 ms
Wall time: 2min 19s


# Still on the command line: Step by step

Here we do the main steps of the conversion.

Every step is a separate run of a python program.
After completion of a step, all information to run a next step, is saved to disk in the form
of result files and report files.

If the results of earlier steps are present, you can just do the following step.

## Step1: Scan ingest

In [9]:
%%time

!python make.py ingest -

Ingest scans ...
CPU times: user 9.15 ms, sys: 7.67 ms, total: 16.8 ms
Wall time: 1.29 s


## Step 2: Scan processing

In [10]:
%%time

!python make.py scans

Process scans ...
CPU times: user 6.13 ms, sys: 7.51 ms, total: 13.6 ms
Wall time: 615 ms


## Step 3: From DOCX to TEI

In [11]:
%%time

!python make.py docx2tei $VERSION

DOCX ==> TEI files ...
DOCX => simple TEI per filza ...
DOCX => headers ...
simple TEI per filza => enriched TEI per letter ...
	There were 542 warnings.
CPU times: user 12.8 ms, sys: 9.18 ms, total: 22 ms
Wall time: 1.66 s


## Step 4: From TEI to TF

In [15]:
%%time

!python make.py tei2tf - $VERSION

TEI => TF ...
	Validating TEI ...
	Converting TEI ...
	Loading TF ...
CPU times: user 322 ms, sys: 119 ms, total: 441 ms
Wall time: 1min 9s


## Step 5: Mark named entities

In [16]:
%%time

!python make.py ner $VERSION

Annotate named entities ...
	Loading TF  ...
5 rows with a duplicate name:
  r305: William, Count of Nassau-Siegen
  r359: William of Orange
  r361: Maurice of Nassau
  r506: Henry II, Duke of Lorraine
  r645: Guillaume III de Melun
1 row without a name:
	e.g.: 365
149 rows without triggers:
	e.g.: 6, 8, 16, 21, 24, 26, 29, 30, 32, 34
Clash: Gugliemo di Nassau: r9 vs r50
Clash: Nicolò Perez: r75 vs r571
Clash: conte di Frusten: r148 vs r149
Clash: conte di Wanderlip: r150 vs r151
Clash: colonello Sciombergh: r253 vs r352
Clash: colonnello Sciombergh: r253 vs r352
Clash: signor di Rocalaura: r386 vs r459
Clash: colonel Rocalaura: r386 vs r459
Clash: monsignor di Rocalaura: r386 vs r459
Clash: colonello Rocalaura: r386 vs r459
	491 entities targeted with 6063 occurrences. See ~/gitlab.huc.knaw.nl/suriano/letters/_temp/ner/0.4.3/.people.0.6/hits.tsv
	Loading TF with entities ...
CPU times: user 306 ms, sys: 107 ms, total: 413 ms
Wall time: 1min 3s


## Step 6: Convert TF to WATM

In [3]:
%%time

!python make.py watm "$VERSION"e
#!python make.py watm "$VERSION"e --no-prod

TF => WATM ...
	Loading TF ...
	Making WATM for version 0.4.3e
	Writing WATM ...
	Testing WATM ...
	OK - whether all tests passed
CPU times: user 45.9 ms, sys: 20.7 ms, total: 66.7 ms
Wall time: 9.3 s


## Step 7: Generate IIIF manifests

### Development

In [4]:
%%time

!python make.py iiif "$VERSION"e
#!python make.py iiif "$VERSION"e --no-prod

Generate IIIF manifests ...
CPU times: user 13.7 ms, sys: 13.1 ms, total: 26.8 ms
Wall time: 2.24 s


## Step 8: Deploy to k8s and TeamText VM

In [ ]:
%%time

!python make.py deploy
#!python make.py deploy --no-prod

# In debugging mode.

Now we dig a bit deeper, en do all the steps while keeping the program in memory.
Now it becomes doable to inspect all intermediate results.

In [1]:
%load_ext autoreload
%autoreload 2

In [44]:
from tf.app import use
from tf.convert.tei import TEI
from tf.convert.addnlp import NLPipeline
from tf.convert.watm import WATM
from tf.convert.iiif import IIIF
from tf.advanced.helpers import dm
from tf.core.files import fileCopy

from processscans import Scans
from processdocs import TeiFromDocx, normalizeChars
from processhelpers import nerMeta, NER_NAME, NERIN_FILE, NEROUT_FILE, SOURCEBASE, PAGESEQ_JSON

## Step 0: Initialization

**N.B.** Check the VERSION variable here!

In [4]:
VERSION = "0.4.4"

## Step 1: Scan ingest

In [5]:
SC = Scans(silent=False, force=False)

In [6]:
SC.ingest(dry=False)

	Already ingested covers. Remove ~/gitlab.huc.knaw.nl/suriano/letters/scans/covers or pass --force to ingest again
	Already ingested pages. Remove ~/gitlab.huc.knaw.nl/suriano/letters/scans/pages or pass --force to ingest again


## Step 2: Scan processing

In [7]:
SC = Scans(silent=False, force=False)
SC.process()

Already present: sizes file originals (covers)
Already present: sizes file originals (pages)
Already present: thumbnails (covers)
Already present: sizes file thumbnails (covers)
Already present: thumbnails (pages)
Already present: sizes file thumbnails (pages)


## Step 3: From DOCX to TEI

You might need to do

```
pip install docx2python
```

In [8]:
TFD = TeiFromDocx(silent=False)

In [9]:
TFD.task("pandoc")

DOCX => simple TEI per filza ...
	02.docx ... uptodate
	03.docx ... uptodate
	04.docx ... uptodate
	05.docx ... uptodate
	06.docx ... uptodate
	07.docx ... uptodate
	08.docx ... uptodate
	09.docx ... uptodate
	10.docx ... uptodate
	11.docx ... uptodate
	12.docx ... uptodate


In [10]:
TFD.task("headers")

DOCX => headers ...
	02.docx
	03.docx
	04.docx
	05.docx
	06.docx
	07.docx
	08.docx
	09.docx
	10.docx
	11.docx
	12.docx
	OK: All headers are OK
Angelo              : 1181 pages in 11 filzas
Cristina            : 1034 pages in 10 filzas
Federica            : 684 pages in  7 filzas
Filippo             : 1084 pages in  9 filzas
Flavia              : 1282 pages in 11 filzas
Giorgia             : 992 pages in  9 filzas
Renzo               :  56 pages in  1 filza 
Ruben               : 1162 pages in 10 filzas
Vera                : 966 pages in 10 filzas
Vera, Federica      : 210 pages in  1 filza 


In [11]:
TFD.task("tei")

  0 x error
8766 x good
	found metadata for 71 letters
simple TEI per filza => enriched TEI per letter ...
	02.xml
	03.xml
	04.xml
	05.xml
	06.xml
	07.xml
	08.xml
	09.xml
	10.xml
	11.xml
	12.xml


0 page warnings



  542 x      no extra letter data in xls. See extrainfo.txt

542 warnings


Pages with    transcription and    scan:  8766
Pages with    transcription and no scan:     0
Pages with no transcription and    scan:     0


See ~/gitlab.huc.knaw.nl/suriano/letters/datasource/transcriptions/report/scantrans.tsv


## Step 4: From TEI to TF

### Check the validity of the TEI.

In [12]:
Tei = TEI(verbose=-1, sourceBase=SOURCEBASE, tei="", tf=VERSION)

In [17]:
Tei.task(check=True, verbose=1, validate=True)

TEI to TF checking: ~/gitlab.huc.knaw.nl/suriano/letters/datasource/tei => ~/gitlab.huc.knaw.nl/suriano/letters/datasource/report
Processing instructions are ignored
XML validation will be performed
Analysing ~/github/annotation/text-fabric/tf/tools/tei/tei_all.xsd
	round   1: 232 changes
INFO: Needs dcr.xsd (exists)
Analysing ~/github/annotation/text-fabric/tf/tools/tei/tei_all.xsd
	round   1: 232 changes
Analysing ~/gitlab.huc.knaw.nl/suriano/letters/datasource/schema/suriano.xsd
	round   1:  68 changes
137 identical override(s)
  0 changing override(s)
Section model I
	Start folder 02:
		   1 suriano                                001.xml                                           
		   2 suriano                                002.xml                                           
		   3 suriano                                003.xml                                           
		   4 suriano                                004.xml                                           
		   5 suriano  

True

### Convert the data

In [18]:
Tei.good = True
Tei.task(convert=True, verbose=0)

Line model II with ln nodes for lines between lb elements
Page model II with page nodes for pages started by pb elements  keeping the pb elements
Section model I
Processing instructions are ignored
Analysing ~/github/annotation/text-fabric/tf/tools/tei/tei_all.xsd
	round   1: 232 changes
Analysing ~/github/annotation/text-fabric/tf/tools/tei/tei_all.xsd
	round   1: 232 changes
Analysing ~/gitlab.huc.knaw.nl/suriano/letters/datasource/schema/suriano.xsd
	round   1:  68 changes
137 identical override(s)
  0 changing override(s)
  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |     0.00s OK
   |     0.00s Following director... 
	Start folder 02:
		   1 suriano                                001.xml                                           
		   2 suriano                                002.xml                                           
		   3 suriano                                003.xml                                           
		   4

True

### Configure a TF app

The TF app has configuration settings, a bit of custom code, and documentation.

Most of it will be generated now, but there are ways to keep custom additions intact.

In [13]:
Tei.task(app=True)

True

### Use the new dataset

The final proof that the conversion has worked is to load the data.
On first-time loading several checks and pre-computations are performed.
Next time the loading will be much quicker.

In [14]:
A = use(f"{Tei.org}/{Tei.repo}:clone", backend=Tei.backend, checkout="clone", silent="verbose", hoist=globals())

**Locating corpus resources ...**

This is Text-Fabric 12.5.3
40 features found and 0 ignored
  0.32s Dataset without structure sections in otext:no structure functions in the T-API
  1.02s All features loaded / computed - for details use TF.isLoaded()
  0.04s All additional features loaded - for details use TF.isLoaded()


Name,# of nodes,# slots / node,% coverage
folder,11,146102.36,100
file,681,2359.95,100
body,681,2281.52,97
text,681,2281.52,97
div,4099,733.15,187
table,243,217.66,3
page,8766,157.23,86
teiHeader,681,78.43,3
chunk,44484,36.13,100
fileDesc,681,48.30,2


In [15]:
n = F.otype.s("pb")[1]
A.plain(n)
A.pretty(n)

## Step 5: Mark named entities

In [16]:
fileCopy(NERIN_FILE, NEROUT_FILE)
NE = A.makeNer(normalizeChars=normalizeChars, silent=False)

In [21]:
# NE.setTask(f".{NER_NAME}.{NER_VERSION}", caseSensitive=True, force=True)
NE.setTask(f".{NER_NAME}", force=True)

Annotation set 🧾 persons has 0 annotations
SHEET data: computing from scratch ...
--------------
Reading sheets
--------------
Sheet with 640 rows and 18 columns


1 row with a duplicate name:
  r481: () Henry II, Duke of Lorraine also in r21



-------------------
Checking scopes ...
-------------------

--
()
--


Ambi: 'gugliemo di nassau': 
  William, Count of Nassau-Siegen: 8
  William Louis, Count of Nassau-Dillenburg: 46
Ambi: 'jacob janson da hoorn': 
  Jacob Meeusen van Hoorn: 431
  Jacob Jansz. van Hoorn: 572
Ambi: 'signor marioni': 
  Pier Antonio Mariano: 448
  Pier Antonio Marioni: 562
Ambiguous triggers: 3 x



--
02
--

--
03
--

--
04
--

--
05
--

--
06
--

--
07
--

--
08
--

--
09
--

---------
10-10.017
---------

-------------
10.018-10.020
-------------

---------
10.021-10
---------

--
11
--

---------
12-12.041
---------

---------
12.042-12
---------
  0.00s Looking up occurrences of many candidates ...
  2.32s done
done


In [22]:
inventory = NE.getSheetData().inventory

In [23]:
triggerData = inventory[('johan.van.groesbek', 'PER')]

In [24]:
triggerData['il baron di grusberch']

{'': [(1498187, 1498188, 1498189, 1498190, 1498191, 1498192)]}

In [25]:
triggerData.get('baron di grusberch', None)

In [26]:
NE.reportHits(showNoHits=True)

Triggers without hits: 157x:


  triggers without hits:


    2da principessa
    [ umena ] figlio
    amsterreadt
    arrigo
    baron di grusberch
    bartoloti
    bernvil
    bilderbeek
    bilderbeke
    brogh
    capitano vassenoven
    cardinal della rosciafucò
    cardinal gleselio
    cent alles
    christiano di bransvich
    collonello vassenoven
    colonello sciomburgh
    consigliero dimer
    conte di ortembourgh
    conte henrico di berghen
    conte nicola governator a verona
    conte nicolo gualdo governatore di verona
    curtlano
    dal presidente di settimana
    dardano
    don luigi di velasco
    duca de wimaer o di sassonia
    duca di bavaria
    duca di holstein
    duca do lunemburg
    duchessa di sassonia
    duci di dupont
    emmanuel filiberto
    ernesto casimiro di nansau
    federigo di solms
    filiberto vernati
    frans boels
    gaffier
    geffier
    geraerdt
    gerbrantssen
    gerritsz .
    giacomo nicheeti
    gio . andrea calandrini
    gio . casimiro di levestein
    gio battista pasini
    

Entities targeted:       623
Triggers searched for:  1320
Triggers without hits:   157
Triggers with hits:     1163
Total hits:             8415

All hits in report file: ~/gitlab.huc.knaw.nl/suriano/letters/_temp/ner/0.4.4/.persons/hits.tsv


In [27]:
NE.findTrigger("baron di grusberch")

**1 occurrence**


In [32]:
NE.makeSheetOfSingleTokens()

In [37]:
NE.setTask(f".{NER_NAME}-single", force=True)

Annotation set 🧾 persons-single has 475521 annotations
SHEET data: computing from scratch ...
--------------
Reading sheets
--------------
Sheet with 1250 rows and 5 columns

-------------------
Checking scopes ...
-------------------

--
()
--
  0.00s Looking up occurrences of many candidates ...
  2.21s done
done


In [38]:
NE.reportHits(showNoHits=True)

Triggers without hits: 71x:


  triggers without hits:


    adriaanson
    alincurt
    amsterreadt
    aranges
    arrigo
    berneuelt
    bernvil
    bilderbeek
    bilderbeke
    boels
    branuich
    brogh
    calandrii
    contareni
    curtlano
    emmanuel
    epinoi
    federigo
    gaffier
    geffier
    geraerdt
    gerbrantssen
    gerritsz
    godfried
    goedefroy
    gormorin
    havert
    hele
    herve
    hillebrandt
    iegerdorf
    iv
    knoop
    ladislaus
    langarauh
    liévin
    lunemburg
    mechior
    mogle
    monsù
    muley
    nicheeti
    ogley
    onderson
    pinoi
    plugen
    proet
    rutghertins
    schiavelinch
    sidan
    silvius
    skyt
    skyte
    solari
    stakenbroeck
    starckenborch
    sterckenborch
    sterkenburch
    sterreburch
    taecke
    taffin
    trever
    udson
    vallore
    vassenoven
    veit
    volbergen
    volrad
    woodhuysen
    zidan
    émile


Entities targeted:      1248
Triggers searched for:  1248
Triggers without hits:    71
Triggers with hits:     1177
Total hits:            475521

All hits in report file: ~/gitlab.huc.knaw.nl/suriano/letters/_temp/ner/0.4.4/.persons-single/hits.tsv


In [39]:
nerMeta(*NE.getMeta(), silent=False)

In [40]:
NE.bakeEntities()

Entity consolidation for 475521 entity occurrences into version 0.4.4e
475521 entity occurrences
  1177 distinct entities
  0.00s Creating a dataset with entity nodes ...
  0.00s preparing and checking ...
  0.00s Feature overview: 37 for nodes; 2 for edges; 1 configs; 9 computed
   |     1.58s done
   |   Delete types: t                   : keep:   shift  nodes       1-1607126 to         1-1607126
   |   Delete types: author              : keep:   shift  nodes 1607127-1607807 to   1607127-1607807
   |   Delete types: bibl                : keep:   shift  nodes 1607808-1608488 to   1607808-1608488
   |   Delete types: biblScope           : keep:   shift  nodes 1608489-1609169 to   1608489-1609169
   |   Delete types: body                : keep:   shift  nodes 1609170-1609850 to   1609170-1609850
   |   Delete types: cell                : keep:   shift  nodes 1609851-1624185 to   1609851-1624185
   |   Delete types: chunk               : keep:   shift  nodes 1624186-1668669 to   1624186-

True

We load the new data:

In [41]:
A = use(f"{Tei.org}/{Tei.repo}:clone", backend=Tei.backend, checkout="clone", silent="verbose", hoist=globals())

**Locating corpus resources ...**

This is Text-Fabric 12.5.3
43 features found and 0 ignored
   |     0.74s T otype                from ~/gitlab.huc.knaw.nl/suriano/letters/tf/0.4.4e
   |       10s T oslots               from ~/gitlab.huc.knaw.nl/suriano/letters/tf/0.4.4e
    11s Dataset without structure sections in otext:no structure functions in the T-API
   |     0.00s T folder               from ~/gitlab.huc.knaw.nl/suriano/letters/tf/0.4.4e
   |     0.07s T chunk                from ~/gitlab.huc.knaw.nl/suriano/letters/tf/0.4.4e
   |     0.00s T file                 from ~/gitlab.huc.knaw.nl/suriano/letters/tf/0.4.4e
   |     3.03s T after                from ~/gitlab.huc.knaw.nl/suriano/letters/tf/0.4.4e
   |     3.58s T str                  from ~/gitlab.huc.knaw.nl/suriano/letters/tf/0.4.4e
   |      |     0.30s C __levels__           from otype, oslots, otext
   |      |       19s C __order__            from otype, oslots, __levels__
   |      |     0.59s C __rank__             from otype, __order__
   |     

Name,# of nodes,# slots / node,% coverage
folder,11,146102.36,100
file,681,2359.95,100
body,681,2281.52,97
text,681,2281.52,97
div,4099,733.15,187
entity,1177,404.84,30
table,243,217.66,3
page,8766,157.23,86
teiHeader,681,78.43,3
chunk,44484,36.13,100


## Step 6: Convert TF to WATM

N.B. For docs click the WATM link in the output cell.

In [42]:
WA = WATM(A, "tei", skipMeta=False, prod=True)
# WA = WATM(A, "tei", skipMeta=False, prod=False)
WA.makeText()
WA.makeAnno()
WA.writeAll()
WA.testAll()

textRepoLevel is section level 'folder'


[WATM exporter documentation](https://annotation.github.io/text-fabric/tf/convert/watm.html)

	Writing WATM ...
Writing production data to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-017/prod
Text file    0:    42233 segments to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-017/prod/text-0.tsv
Text file    1:   114086 segments to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-017/prod/text-1.tsv
Text file    2:   143568 segments to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-017/prod/text-2.tsv
Text file    3:    98423 segments to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-017/prod/text-3.tsv
Text file    4:   143925 segments to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-017/prod/text-4.tsv
Text file   10:   121885 segments to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-017/prod/text-10.tsv
Text files all:  1607126 segments to 11 files
Anno file    1:   400000 annotations written to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-017/prod/anno-1.tsv
Anno file    2:   400000 annotations written to ~/gitlab.huc.knaw.nl/suriano/letters/watm/0.4.4e-

## Step 7: Generate IIIF manifests

In [45]:
II = IIIF(Tei.teiVersion, A, PAGESEQ_JSON, prod=True, silent=False)
# II = IIIF(Tei.teiVersion, A, prod=False, silent=False)
II.manifests()

Maximum dimensions: W = 8504 H = 5976
Average dimensions: W = 4138 H = 4869
Average deviation:  W = 1038 H =  660
Maximum dimensions: W = 5600 H = 5786
Average dimensions: W = 3253 H = 4359
Average deviation:  W =  477 H =  607
Collections:
   02 with  262 pages
   03 with  660 pages
   04 with  806 pages
   05 with  688 pages
   06 with  684 pages
   07 with  628 pages
   08 with  988 pages
   09 with  944 pages
   10 with 1062 pages
   11 with 1116 pages
   12 with  928 pages
IIIF manifests generated in ~/gitlab.huc.knaw.nl/suriano/letters/static/prod/manifests


## Step 8: Deploy to k8s and TeamText VM

NB: you need to have access to the k8s cluster and to the team text VM.

That means:

* The LDAP of the relevant k8s clusters know you
* You have an ssh key-based login on the Team Text VPN
* You work inside the firewall

In [53]:
!./provision.sh prod watm
#!./provision.sh dev watm

k-suite enabled
Context "k8s-10-26-6-0" modified.

Quick access to iiif-suriano : type khelp for an overview of commands.

anno-1.tsv                                    100%   16MB   6.2MB/s   00:02    
anno-3.tsv                                    100%   15MB   5.3MB/s   00:02    
anno-2.tsv                                    100%   13MB   5.1MB/s   00:02    
anno-6.tsv                                    100%   10MB   5.3MB/s   00:01    
anno-5.tsv                                    100%   15MB   4.7MB/s   00:03    
anno-4.tsv                                    100%   15MB   5.7MB/s   00:02    
anno2node.tsv                                 100%   14MB   5.7MB/s   00:02    
text-6.tsv                                    100% 1135KB   5.8MB/s   00:00    
text-7.tsv                                    100% 1088KB   4.2MB/s   00:00    
text-5.tsv                                    100%  951KB   5.7MB/s   00:00    
text-4.tsv                                    100%  852KB   6.4MB/s   00:00  

In [48]:
!./provision.sh prod files
#!./provision.sh dev files

k-suite enabled
Context "k8s-10-26-6-0" modified.

Quick access to iiif-suriano : type khelp for an overview of commands.

copying to pod: prod/covers.html
copying to pod: prod/logo
copying to pod: prod/manifests
copying to pod: both/metadata


In [ ]:
!./provision.sh prod images
#!./provision.sh dev images

## Step 9: Test the images

* [covers](https://suriano.huygens.knaw.nl/files/covers.html)

* [02.json](https://suriano.huygens.knaw.nl/files/manifests/02.json)

* [page 02_171r](https://suriano.huygens.knaw.nl/iiif/3/pages%2F02_071r.jpg/full/max/0/default.jpg)